## ANSWERING BUSINESS QUESTIONS USING SQL

In this project, we're going to use SQL skills to answer questions pertaining to a fictional digital music business.  The database that we'll access is called "chinook.db".  Here is a description of a few of the tables in this database:  (1) employee - this table lists pertinent personnel information such as the employee ID, their full names, addresses, hire dates, etc.; (2) customer - this summarizes info pertaining to customer ID, full names, addresses, and the sales representative that is assigned to each customer; (3) invoice - tracks all the invoices processed which include the customer who made the purchase, location where the purchase was made, and the invoice total; (4) invoice line - tracks all the details for each of the invoices as it lists all the individual music tracks purchased per each invoice, the price of the track, and quantity bought; (5) track - provides specific info about each track such as track ID, name, album ID, genre type, etc.; (6) album - gives detailed info on the album ID, name, and the artist who produced the album.

We'll be using SQL queries to determine things such as the number of tracks sold per each genre in the USA, the total dollar amount of sales made by each sales support agent in the company, and which invoice happens to be a full album purchase vs. individual track purchase.

In [1]:
import sqlite3
import os
import pandas as pd

In [2]:
# connect to the chinook database
conn = sqlite3.connect('chinook.db')

### Overview of the data

In [3]:
# write a query to return information on the tables and views in the database
cursor1 = conn.execute("SELECT name, type FROM sqlite_master WHERE type IN ('table', 'view')")
db = []
for row in cursor1:
    db.append(row)
table1 = pd.DataFrame(db, columns = ['name', 'type'])

print(table1)

              name   type
0            album  table
1           artist  table
2         customer  table
3         employee  table
4            genre  table
5          invoice  table
6     invoice_line  table
7       media_type  table
8         playlist  table
9   playlist_track  table
10           track  table


### Selecting new albums to purchase

In [4]:
# write a query that returns each genre, with the number of tracks sold in the USA in absolute numbers and in percentages

cursor2 = conn.execute("WITH usa_tracks_sold AS \
        (SELECT il.* FROM invoice_line il \
         INNER JOIN invoice i on il.invoice_id = i.invoice_id \
         INNER JOIN customer c on i.customer_id = c.customer_id \
         WHERE c.country = 'USA') \
        SELECT g.name genre, \
        count(uts.invoice_line_id) tracks_sold, \
        cast(count(uts.invoice_line_id) AS FLOAT) / (SELECT COUNT(*) from usa_tracks_sold) \
        percentage_sold \
        FROM usa_tracks_sold uts \
        INNER JOIN track t on t.track_id = uts.track_id \
        INNER JOIN genre g on g.genre_id = t.genre_id \
        GROUP BY 1 \
        ORDER BY 2 DESC \
        LIMIT 10")  
db1 = []
for row in cursor2:
    db1.append(row)
table2 = pd.DataFrame(db1, columns = ['genre', 'tracks sold', 'percentage sold'])

print(table2)

                genre  tracks sold  percentage sold
0                Rock          561         0.533777
1  Alternative & Punk          130         0.123692
2               Metal          124         0.117983
3            R&B/Soul           53         0.050428
4               Blues           36         0.034253
5         Alternative           35         0.033302
6                 Pop           22         0.020932
7               Latin           22         0.020932
8         Hip Hop/Rap           20         0.019029
9                Jazz           14         0.013321


Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:

Red Tone (Punk);
Slim Jim Bites (Blues);
Meteor and the Girls (Pop)

It's worth keeping in mind that combined, these three genres only make up about 17% of the total sales, so we should be on the lookout for artists and albums from the 'rock' genre, which accounts for 53% of sales.

### Analyzing Employee Sales Performance

In [5]:
# write a query that finds the total dollar amount of sales assigned to each sales support agent within the company
cursor3 = conn.execute("WITH customer_support_rep_sales AS \
                      (SELECT i.customer_id, c.support_rep_id, \
                      SUM(i.total) total FROM invoice i \
                      INNER JOIN customer c ON i.customer_id = c.customer_id \
                      GROUP BY 1,2) \
                      SELECT \
                      e.first_name|| ' ' ||e.last_name employee, \
                      e.hire_date, SUM(csrs.total) total_sales \
                      FROM customer_support_rep_sales csrs \
                      INNER JOIN employee e ON e.employee_id = csrs.support_rep_id \
                      GROUP BY 1")
db2 = []
for row in cursor3:
    db2.append(row)
table3 = pd.DataFrame(db2, columns = ['employee', 'hire_date', 'total_sales'])

print(table3)

        employee            hire_date  total_sales
0   Jane Peacock  2017-04-01 00:00:00      1731.51
1  Margaret Park  2017-05-03 00:00:00      1584.00
2  Steve Johnson  2017-10-17 00:00:00      1393.92


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates. 

### Analyzing sales by country

In [6]:
# write a query that collates data on purchases from different countries; for each country, report the total 
# no. of customers, total value of sales, average value of sales per customer, and the average order value

cursor4 = conn.execute("WITH country_or_other AS \
                      (SELECT \
                           CASE \
                               WHEN (SELECT count(*) \
                                     FROM customer \
                                     where country = c.country) = 1 THEN 'Other' \
                                     ELSE c.country \
                                     END AS country, \
                                     c.customer_id, il.* \
                                     FROM invoice_line il \
                                     INNER JOIN invoice i ON i.invoice_id = il.invoice_id \
                                     INNER JOIN customer c ON c.customer_id = i.customer_id) \
                       SELECT \
                       country, customers, total_sales, average_order, customer_lifetime_value \
                       FROM \
                       (SELECT \
                       country, count(distinct customer_id) customers, \
                       SUM(unit_price) total_sales, \
                       SUM(unit_price) / count(distinct customer_id) customer_lifetime_value, \
                       SUM(unit_price) / count(distinct invoice_id) average_order, \
                           CASE \
                               WHEN country = 'Other' THEN 1 \
                               ELSE 0 \
                               END AS sort \
                               FROM country_or_other \
                               GROUP BY country \
                               ORDER BY sort ASC, total_sales DESC)")
db3 = []
for row in cursor4:
    db3.append(row)
table4 = pd.DataFrame(db3, columns = ['country', 'customers', 'total_sales', 'averaged_order', 'customer_lifetime_value'])

print(table4)

          country  customers  total_sales  averaged_order  \
0             USA         13      1040.49        7.942672   
1          Canada          8       535.59        7.047237   
2          Brazil          5       427.68        7.011148   
3          France          5       389.07        7.781400   
4         Germany          4       334.62        8.161463   
5  Czech Republic          2       273.24        9.108000   
6  United Kingdom          3       245.52        8.768571   
7        Portugal          2       185.13        6.383793   
8           India          2       183.15        8.721429   
9           Other         15      1094.94        7.448571   

   customer_lifetime_value  
0                80.037692  
1                66.948750  
2                85.536000  
3                77.814000  
4                83.655000  
5               136.620000  
6                81.840000  
7                92.565000  
8                91.575000  
9                72.996000  


Based on the data, there may be opportunity in the following countries:

Czech Republic,
United Kingdom,
India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low, we should be cautious  about spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the data from new customers to make sure that these trends still hold.

### Albums vs individual tracks

In [7]:
# write a query that categorizes each invoice as either a full album purchase or not
cursor5 = conn.execute("WITH invoice_first_track AS \
                      (SELECT \
                      il.invoice_id invoice_id, MIN(il.track_id) first_track_id \
                      FROM invoice_line il \
                      GROUP BY 1) \
                      SELECT album_purchase, \
                      COUNT(invoice_id) number_of_invoices, \
                      CAST(count(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM invoice) percent \
                      FROM \
                      (SELECT ifs.*, \
                      CASE \
                           WHEN \
                     (SELECT t.track_id FROM track t \
                      WHERE t.album_id = (SELECT t2.album_id FROM track t2 \
                                          WHERE t2.track_id = ifs.first_track_id) \
                     EXCEPT \
                     SELECT il2.track_id FROM invoice_line il2 \
                     WHERE il2.invoice_id = ifs.invoice_id) IS NULL \
                     AND \
                     (SELECT il2.track_id FROM invoice_line il2 \
                     WHERE il2.invoice_id = ifs.invoice_id \
                     EXCEPT \
                     SELECT t.track_id FROM track t \
                     WHERE t.album_id = (SELECT t2.album_id FROM track t2 \
                                         WHERE t2.track_id = ifs.first_track_id)) IS NULL \
                     THEN 'yes' \
                     ELSE 'no' \
                     END AS 'album_purchase' \
                     FROM invoice_first_track ifs) \
                     GROUP BY album_purchase")
db4 = []
for row in cursor5:
    db4.append(row)
table5 = pd.DataFrame(db4, columns = ['album_purchase', 'number_of_invoices', 'percent']) 

print(table5)

  album_purchase  number_of_invoices   percent
0             no                 500  0.814332
1            yes                 114  0.185668


Album purchases account for as much as 18.6% of all the purchases! Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of the revenue.